# Relatório GPU

Supercomputação: Cicero Tiago

### 1 Algoritmo de Smith-Waterman + Busca Exaustiva

Uma cálculo simples do _score_ utilizando o algoritmo de Smith-Waterman pode ser 
definido como a seguir, sendo $S_{i,j}$ o _score_ do elemento posicionado na linha 
$i$ e coluna $j$ da matriz utilizada para o cálculo:

$$ 
S_{i,j} = max\begin{Bmatrix}
S_{i-1, j-1} + 2, & a_i = b_j \\ 
S_{i-1, j-1} - 1,  & a_i \neq b_j\\ 
S_{i-1, j} - 1 &  b_j = -\\
S_{i, j-1} - 1 &  a_i = -\\ 
0 & 
\end{Bmatrix}
$$ 

Como podemos ver, o cálculo do score de um elemento só depende do score de elementos 
imediatamente anteriores. Com isso, esse cálculo pode ser dividido em duas etapas:

#### 1.1 Primeira Etapa

Depende apenas do score dos elementos na **linha** anterior.

$$ 
S_{temp}(i,j) = max\begin{Bmatrix}
S_{i-1, j-1} + 2, & a_i = b_j \\ 
S_{i-1, j-1} - 1,  & a_i \neq b_j\\ 
S_{i, j-1} - 1 &  b_j = -\\
0 & 
\end{Bmatrix}
$$ 

#### 1.2 Segunda Etapa:

Depende apenas do cálculo da etapa anterior e do score do elemento na coluna anterior.

$$ 
S_{i,j} = max\begin{Bmatrix}
S_{temp}(i,j) & \\ 
S_{temp}(i-1, j) - 1 &  \\
0 & 
\end{Bmatrix}
$$

### 2 Implementação

#### 2.1 Leitura do arquivo

Ler o arquivo de entrada que segue um padrão de acordo com a descrição a seguir:\
**L1**: inteiro N representando o tamanho da primeira sequência \
**L2**: inteiro M representando o tamanho da segunda sequência \
**L3**: sequencia 1 de tamanho N \
**L4**: sequencia 2 tamanho M

Cada sequência do arquivo de entrada pode então ser guardada em um `std::vector<char>`. Chamaremos a sequência de tamanho N de A e a de tamanho M de B.

#### 2.2 Gerar subsequências

Uma vez que cada subsequência representa um conjunto sequencial de caracteres contidos na sequência maior, cada uma delas pode ser definida por dois números inteiros, aqui chamados de `start` e `length`. 

Sendo assim, pode ser definida uma função que recebe 3 argumentos (sequência, ponto de início e tamanho) e devolve a subsequência do tipo `std::string`.

```cpp
std::string get_subsequence(std::string String, int start, int length) {
  return String.substr(start, length);
}
```

Agora que é possível gerar subsequências, podemos gerar todas as subsequências de A, todas as subsequências de B e cruzar todas contra todas. Analisando quantitativamente, o número de comparações cresce bem rapidamente:

Seja A uma sequência de 4 bases nitrogenadas: ACCA;\
Seja B uma sequência de 4 bases nitrogenadas: GTGT;

As subsequências de A são 10: A C C A AC CC CA ACC CCA ACCA;\
As subsequências de B são 10: G T G T GT TG GT GTG TGT GTGT;

Com isso, para realizar a busca exaustiva entre A e B, cada uma de tamanho 4, seriam necessárias 10 vezes 10 comparações. Para essa etapa, as subsequências foram armazenadas em um `std::vector` (alocado na memória do host).

Para gerar as subsequências, foi utilizada a seguinte estrutura que utiliza paralelização com _OpenMP_:

```cpp
// A e B são sequências
// N e M são seus respectivos tamanhos
// SA e SB são seus respectivos arrays de subsequências
#pragma omp parallel
{
  #pragma omp master
  {
    #pragma omp task shared(SA)
    for (int length = 1; length <= N; length++) {
      for (int start = 0; start < N - length + 1; start++) {
        SA.push_back(get_subsequence(A, start, length));
      }
    }
    #pragma omp task shared(SB)
    for (int length = 1; length <= M; length++) {
      for (int start = 0; start < M - length + 1; start++) {
        SB.push_back(get_subsequence(B, start, length));
      }
    }
  }
}
```

#### 2.3 Comparações

##### 2.3.1 Functor

A seguir está descrito o functor responsável por _computar score_. 
A estratégia é a seguinte:

- O **functor** se assemelha ao recurso de [_currying_](https://www.treinaweb.com.br/blog/conceitos-de-linguagens-funcionais-o-que-e-currying)
dado que, uma vez que a struct é instanciada com os parâmetros `_dS`, `_chr` e `_previous_row`,
as operações realizada por essa instância terão acesso a esses valores;
- O construtor recebe `_dS` (conjunto de caracteres da segunda subsequência), 
`_char` (caractere da primeira subsequência) e `_previous_row` (linha anterior da matriz já preenchida);
- O operador da _struct_ recebe um inteiro representando o índice atual $i$ e, a partir disso, 
calcula o score para os casos de _match_, _gap_ e _mismatch_;


```cpp
struct Compute {
  thrust::device_ptr<char> dS;
  thrust::device_ptr<int> previous_row;
  char chr;

  Compute(thrust::device_ptr<char> _dS, char _chr,  thrust::device_ptr<int> _previous_row)
      : dS(_dS), chr(_chr), previous_row(_previous_row) {};

  __host__ __device__ int operator()(const int(&i)) {
    int score;
    char comparing_char = dS[i];

    if (chr == comparing_char)
      score = previous_row[i - 1] + WMAT;
    else if (chr == '-' || comparing_char == '-')
      score = previous_row[i - 1] + WGAP;
    else
      score = previous_row[i - 1] + WMIS;

    return score > 0 ? score : 0;
  }
};
```


#### 2.3.2 Função que Compara Subsequências

Para realizar as comparações entre duas subsequências, utilizaremos seguinte função 
que pode ser dividida no seguinte conjunto de passos:

- Inicializa e preenche a primeira linha da matriz com zeros;
- Utiliza a ferramenta [**`thrust::transform`**](https://thrust.github.io/doc/group__transformations_gacbd546527729f24f27dc44e34a5b8f73.html) que aplica um 
utiliza como argumento do operador unário retornado pelo functor cada iterator 
`*i` entre `c0` e `c1` e armazena o resultado endereço corresponde to terceiro 
argumento `current_row.begin() + 1`. Essa etapa representa o item 1.1;
- Em seguida, utiliza [**`thrust::inclusive_scan`**](https://thrust.github.io/doc/group__prefixsums_gaaa5aa56f22c5e74e55ffdfebda8fbb62.html) para aplicar o __operador binário associativo__
  `thrust::maximum`, implementando o item 1.2;

```cpp
int subsequences_score(const std::string ssA, const std::string ssB){
  const int N = ssA.size();
  const int M = ssB.size();

  thrust::device_vector<int> previous_row(N + 1);
  thrust::device_vector<int> current_row(N + 1);

  previous_row.resize(N + 1);
  current_row.resize(N + 1);

  thrust::fill(previous_row.begin(), previous_row.end(), 0);

  thrust::device_vector<char> dS(N);
  thrust::copy(ssA.begin(), ssA.begin() + N, dS.begin());

  thrust::counting_iterator<int> c0(1);
  thrust::counting_iterator<int> c1(M + 1);

  for (int i = 0; i < M; i++) {
    char comparing_char = ssB[i];
    thrust::transform(c0, c1, current_row.begin() + 1, Compute(dS.data(), comparing_char, previous_row.data()));
    thrust::inclusive_scan(current_row.begin() + 1, current_row.end(), previous_row.begin() + 1, thrust::maximum<int>());
  }

  return thrust::reduce(current_row.begin() + 1, current_row.end(), -1, thrust::maximum<int>());
}
```

#### 2.3.3 Laço de Comparação

A etapa aqui chamada de laço de comparação é responsável por selecionar
 subsequências em `SA` e `SB` obter o score entre elas utilizando a função
 `subsequences_score` definida anteriormente. Para isso, foi utilizano OpenMP para
 paralelizar a busca pelo melhor score.

```cpp
int max_score = 0;

#pragma omp parallel shared(max_score)
{
  #pragma omp for reduction(max : max_score)
  for (int index = 0; index < SA.size() * SB.size(); index++) {
    // truque para iterar nas duas listas utilizando um único iterador
    int indexA = (int) index / SB.size();
    int indexB = (int) index % SB.size();

    std::string ssA = SA.at(indexA);
    std::string ssB = SB.at(indexB);

    int local_score = subsequences_score(ssA, ssB);

    if (local_score > max_score) {
      #pragma omp critical
      max_score = local_score;
    }
  }
}
```

### 3 Resultados


#### 3.1 Geração de subsequências

Utilizando o computador "monstrão" e sequências de tamanhos razoáveis
 (sequência de tamanho 100 gerando 5.5k subsequências), gastava algumas frações
 de milisegundos.

#### 3.2 Comparação

Essa etapa foi onde os surgiram os problemas. Para sequências pequenas (tamanho 10)
 já levava mais de 1 segundo e aumentando pouco a pouco a sequência de entrada,
 esse tempo chegava facilmente a dezenas de segundos.

#### 3.2.1 Possíveis falhas

O item 2.3.3 lê as subsequências da memória do host e transfere, mesmo que
 utilizando paralelização, para a função `subsequences_score` que, por sua vez,
 aloca vetores `thrust::device_vector`, preenche com zeros etc. e transfere 
 subsequências para a GPU. Para sequências de tamanho maior que 32 (1k subsequências),
 essas operações já são realizadas mais de 1 milhão de vezes, o que pode o 
 prejuízo na performance.

#### 3.2.2 Proposta de solução

Uma possível melhoria para o tempo poderia ser alterar o functor para receber as **sequências** no construtor
 e o operador poderia ser responsável por gerar comparar 1 `char` de cada conforme a necessidade.
 Com isso, o construtor da `struct` seria chamado fora do loop, certamente tornando mais econômico em processamento.